<div align="right">
Massimo Nocentini<br>
<small>
<br>September 21, 2016: Monge arrays
</small>
<br>
<img src="http://www.cerm.unifi.it/chianti/images/logo%20unifi_positivo.jpg" 
        alt="UniFI logo" style="float: right; width: 20%; height: 20%;">
</div>
<br>
<div align="center"><br>
<b>Abstract</b><br>
This document collects ideas about <i>dynamic programming</i> and related concepts,
such as recurrence relations and matrices.
</div>

# Matrices

## Monge arrays

Have a look at https://en.wikipedia.org/wiki/Monge_array

In [69]:
import operator
from itertools import count, zip_longest
from collections import OrderedDict

def parity_numbered_rows(matrix, parity, include_index=False):
    start = 0 if parity == 'even' else 1
    return [(i, r) if include_index else r 
            for i in range(start, len(matrix), 2)
            for r in [matrix[i]]]
    
def argmin(iterable, only_index=True):
    index, minimum = index_min = min(enumerate(iterable), key=operator.itemgetter(1))
    return index if only_index else index_min

def interleaving(one, another):
    for o, a in zip_longest(one, another):
        yield o
        if a: yield a

def is_sorted(iterable, pred=lambda l, g: l <= g):
    _, *rest = iterable
    return all(pred(l, g) for l, g in zip(iterable, rest))
            
def minima_indexes(matrix):
    
    if len(matrix) == 1: return [argmin(matrix.pop())]
    
    recursion = minima_indexes(parity_numbered_rows(matrix, parity='even'))
    even_minima = OrderedDict((i, m) for i, m in zip(count(start=0, step=2), recursion))
    odd_minima = [argmin(odd_r[start:end]) + start
                  for o, odd_r in parity_numbered_rows(matrix, parity='odd', include_index=True)
                  for start in [even_minima[o-1]]
                  for end in [even_minima[o+1]+1 if o+1 in even_minima else None]]
    
    return list(interleaving(even_minima.values(), odd_minima))

def minima(matrix):
    return [matrix[i][m] for i, m in enumerate(minima_indexes(matrix))]

def is_not_monge(matrix):
    return any(any(matrix[r][m] > matrix[r][i] for i in range(m)) 
               for r, m in enumerate(minima_indexes(matrix)))
    

The following *is* a Monge array:

In [70]:
matrix = [
    [10, 17, 13, 28, 23],
    [17, 22, 16, 29, 23],
    [24, 28, 22, 34, 24],
    [11, 13, 6, 17, 7],
    [45, 44, 32, 37, 23],
    [36, 33, 19, 21, 6],
    [75, 66, 51, 53, 34],
]

minima(matrix)

[10, 16, 22, 6, 23, 6, 34]

In [71]:
minima_indexes(matrix)

[0, 2, 2, 2, 4, 4, 4]

In [72]:
is_not_monge(matrix)

False

The following *is not* a Monge array:

In [73]:
matrix = [
    [37, 23, 22, 32],
    [21, 6, 7, 10],
    [53, 34, 30, 31],
    [32, 13, 9, 6],
    [43, 21, 15, 8],
]

minima(matrix) # produces a wrong answer!!!

[22, 7, 30, 6, 8]

In [74]:
minima_indexes(matrix)

[2, 2, 2, 3, 3]

In [75]:
is_not_monge(matrix)

True

---
<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.